# Query - Keyword test

- 단순 쿼리 유사도로 시작, 끝 노드를 찾기에는 어려움이 있음
- keyword를 어떻게 활용할 것인가?

In [6]:
import json, os

from openai import OpenAI
from dotenv import load_dotenv

CONFIG_PATH = "configs/config.json"


# config_file 가져오기
with open(CONFIG_PATH, "r", encoding="utf-8") as f:
    config = json.load(f)
    print(f"Loaded config data from {CONFIG_PATH}")


# 프롬프트 불러오기
with open("./GraphDB/prompt/get_query_keyword.txt", 'r', encoding='utf-8') as file:
    USER_KEYWORD_EXTRACT_PROMPT = file.read()

load_dotenv(verbose=True)
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

Loaded config data from configs/config.json


In [14]:
# Query Extraction
def extract_user_keyword(query):
    completion = client.chat.completions.create(
            model=config['model'],
            messages=[
                {"role": "system", "content": USER_KEYWORD_EXTRACT_PROMPT},
                {
                    "role": "user",
                    "content": query
                }
            ]
    )

    print(f"## Query : {query}/n/nKeyword : {completion.choices[0].message.content}")

    answer = completion.choices[0].message.content
    answer = [x.strip() for x in answer.split("|")]
    return answer

In [15]:
QUERY_1 = "정정신고서를 부실기재 했을 경우 과징금은 얼마이고, 과징금이 부과되는 대상은 누구인가?"

In [20]:
ANSWER_1 = extract_user_keyword(QUERY_1)

## Query : 정정신고서를 부실기재 했을 경우 과징금은 얼마이고, 과징금이 부과되는 대상은 누구인가?/n/nKeyword : 정정신고서 | 부실기재 | 과징금 | 부과 대상


In [21]:
ANSWER_1

['정정신고서', '부실기재', '과징금', '부과 대상']

## nC2

In [23]:
from itertools import combinations as comb


ANSWER_list = list(comb(ANSWER_1, 2))
ANSWER_list = [x[0] + ' ' + x[1] for x in ANSWER_list]
ANSWER_list

['정정신고서 부실기재',
 '정정신고서 과징금',
 '정정신고서 부과 대상',
 '부실기재 과징금',
 '부실기재 부과 대상',
 '과징금 부과 대상']

## KeywordGraph

In [ ]:
from GraphDB.LegalGraphDB import LegalGraphDB


dbms = LegalGraphDB(auradb=False, config=config, json_path="../data/graph/clause/title_embedding/01/01_law_main.json")